In [2]:
import pandas as pd
import psycopg2
import os
import glob

In [3]:
def get_latest_file(directory):
    """
    Get latest file from directory.
    
    :param dicrectory: The dicrectory to save data.
    :return: List for data dictionary.
    """

    # Get a list of all files in the directory
    files = glob.glob(os.path.join(directory, "*.json"))
    
    # Check if there are any files in the directory
    if not files:
        print(f"The directory {directory} is empty or contains no files.")
        return None
    
    # Get the latest file based on modification time
    latest_file = max(files, key=os.path.getmtime)
    return latest_file

In [4]:
path_companies = '/home/ngocthang/Documents/Code/Stock-Company-Analysis/backend/data/transformed/transformed_to_database_companies'
path_exchanges = '/home/ngocthang/Documents/Code/Stock-Company-Analysis/backend/data/transformed/transformed_to_database_exchanges'
path_industries = '/home/ngocthang/Documents/Code/Stock-Company-Analysis/backend/data/transformed/transformed_to_database_industries'
path_regions = '/home/ngocthang/Documents/Code/Stock-Company-Analysis/backend/data/transformed/transformed_to_database_regions'
path_sic_industries = '/home/ngocthang/Documents/Code/Stock-Company-Analysis/backend/data/transformed/transformed_to_database_sic_industries'

In [5]:
file = get_latest_file(path_companies)
print(file)

/home/ngocthang/Documents/Code/Stock-Company-Analysis/backend/data/transformed/transformed_to_database_companies/transformed_companies_2024_06_19.json


In [6]:
df = pd.read_json(file, lines=True)
df.head()

,company_exchange,company_industry,company_sector,company_sic_id,company_name,company_ticket,company_is_delisted,company_category,company_currency,company_location
0,NASDAQ,Banks - Regional,Financial Services,6022,ADMIRALTY BANCORP INC,AAAB,True,Domestic Common Stock,USD,Florida; U.S.A
1,NASDAQ,Banks - Regional,Financial Services,6035,ACCESS ANYTIME BANCORP INC,AABC,True,Domestic Common Stock,USD,New Mexico; U.S.A
2,NASDAQ,Asset Management,Financial Services,6153,ASSET ACCEPTANCE CAPITAL CORP,AACC,True,Domestic Common Stock,USD,Michigan; U.S.A
3,NASDAQ,Credit Services,Financial Services,6099,ACE CASH EXPRESS INC,AACE,True,Domestic Common Stock,USD,Texas; U.S.A
4,NASDAQ,Shell Companies,Industrials,6770,ARMADA ACQUISITION CORP I,AACI,False,Domestic Common Stock Primary Class,USD,Pennsylvania; U.S.A


In [12]:
columns = list(df.columns)
print(columns)

['company_exchange', 'company_industry', 'company_sector', 'company_sic_id', 'company_name', 'company_ticket', 'company_is_delisted', 'company_category', 'company_currency', 'company_location']


In [13]:
placeholders = ', '.join(['%s'] * len(columns))
columns_str = ', '.join(columns)

In [14]:
print(placeholders)

%s, %s, %s, %s, %s, %s, %s, %s, %s, %s


In [15]:
print(columns_str)

company_exchange, company_industry, company_sector, company_sic_id, company_name, company_ticket, company_is_delisted, company_category, company_currency, company_location


In [45]:

# Read the SQL file
path = "/home/ngocthang/Documents/Code/Stock-Company-Analysis/backend/scripts/load/sql-scripts/upsert.sql"
with open(path, 'r') as file:
    sql_script = file.read()

In [40]:
print(sql_script[3])

S


In [41]:
b = sql_script.split(';')

In [46]:
for statement in sql_script.split(';'):
    print(statement)


INSERT INTO regions(region_name, region_local_open, region_local_close)
    SELECT 
        * 
    FROM temp_regions
ON CONFLICT (region_name) DO NOTHING


INSERT INTO industries(industry_name, industry_sector)
    SELECT
        *
    FROM temp_industries
ON CONFLICT (industry_name, industry_sector) DO NOTHING


INSERT INTO sic_industries(sic_id, sic_industry, sic_sector)
    SELECT
        * 
    FROM temp_sic_industries
ON CONFLICT (sic_industry, sic_sector) DO NOTHING




INSERT INTO exchanges(exchange_region_id, exchange_name)
    SELECT
        r.region_id,
        e.exchange_name
    FROM temp_exchanges e
    JOIN regions r ON r.region_name = e.region
ON CONFLICT (exchange_name) DO NOTHING



INSERT INTO companies(company_id, company_exchange_id, company_industry_id, company_sic_id, company_name, company_ticket, company_is_delisted, company_category, company_currency, company_location)
    SELECT 
        c.company_id,
        e.exchange_id as company_exchange_id,
        i.ind